In [1]:
# 一通り動かしてみる

#ライブラリの自動インポート
%reload_ext autoreload
%autoreload 2

In [12]:
import pandas as pd
import openai
from tqdm import tqdm
from src.utils.GPT import json_generate
from src.qra_gen import generate_reason_and_predict

In [3]:
#api_key
from api_key import api_key
openai.api_key = api_key

In [4]:
#laod dataset as dict
csv_path="dataset/BradleyMeltingPointDataset_practice.csv"
df=pd.read_csv(csv_path)
chemical_records=df.to_dict(orient='records')
chemical_records[:3]

[{'mpC': 152.0,
  'name': '3-phthalimidopropionic acid',
  'smiles': 'c1ccc2c(c1)C(=O)N(C2=O)CCC(=O)O',
  'csid': 69310,
  'link': 'http://www.alfa.com/en/GP100W.pgm?DSSTK=L13535',
  'source': 'Alfa Aesar'},
 {'mpC': 33.5,
  'name': '1-naphthalenecarboxaldehyde',
  'smiles': 'O=Cc2cccc1ccccc12',
  'csid': 5960,
  'link': 'http://www.srcinc.com/what-we-do/product.aspx?id=133',
  'source': 'PHYSPROP'},
 {'mpC': 214.0,
  'name': 'tabernanthine',
  'smiles': 'COc4cc5nc1c(CCN3CC2CC1C3C(C2)CC)c5cc4',
  'csid': 4265369,
  'link': 'http://www.srcinc.com/what-we-do/product.aspx?id=133',
  'source': 'PHYSPROP'}]

In [5]:
#このプロンプトを変更

# 質問文を生成するための指示
qa_gen_command="""
Provide the quantitative reasons within 300 words so that a scientist, who does not know the melting point, can predict the value.
We must quantitatively consider how the melting point shifts.
I absolutely forbid you to make qualitative generalizations.

#Bad example reasons
## Its molecular weight compared to simpler aromatic compounds, contributing to a higher melting point. (qualitative discussion is practically meaningless!!!)
## Therefore, the compound has a melting point of 110°C (Never include the answer in the reason!!).

#Good example reasons
## Benzene has a boiling point of 80 degrees, but toluene with a methyl group improves the boiling point by about +30 degrees due to its larger molecular weight and other reasons (+30 contribution).
## Butane has a boiling point of -1°C, but butanol with a hydroxy group has a boiling point increase of about 115°C due to the strong influence of hydrogen bonding (+115 contribution).

#Output: Reason key
"""

# 分子構造から物性を予測するための指示
predict_command="""
Predict the melting point of the following compound.
In any case, output some value.

#Output: Value key
"""

#use GPT3.5 completion
llm_ask_func=json_generate

#if gen_reason=True, generate reason and predict
#if gen_reason=False, only predict without reason (for control)
gen_reason=True
gen_reason=False

In [16]:


completed_records=[]
for record in tqdm(chemical_records):
    try:
        completed_record=generate_reason_and_predict(record["name"],record["mpC"],
                                        qa_gen_command,
                                        predict_command,
                                        llm_ask_func=json_generate,
                                        smiles=record["smiles"],
                                        gen_reason=gen_reason,
                                        n_trials=3)
    except Exception as e:
        print(e)
        completed_record=record

    completed_records.append(completed_record)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:06,  1.36it/s]

'Value'


 90%|█████████ | 9/10 [00:13<00:01,  1.39s/it]

'Value'


100%|██████████| 10/10 [00:14<00:00,  1.45s/it]

'Value'


In [17]:

import datetime
import json

study_name="1216_practice10"
now=datetime.datetime.now()
now_str=now.strftime("%Y%m%d_%H%M%S")
save_path=f"results/{study_name}_{now_str}.json"
with open(save_path, 'w') as f:
    json.dump(completed_records, f, indent=4, ensure_ascii=False)